In [ ]:
%pip install accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 trl==0.7.10

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
from transformers import AutoTokenizer, LlamaForCausalLM
import torch
import argparse
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "squarelike/llama2-ko-medical-7b"

compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quant_config,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/sseungtest1/checkpoint-523")

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
logging.set_verbosity(logging.CRITICAL)

prompt = "트리플 바이패스 수술"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[QUESTION] {prompt} [/QUESTION]")
print(result[0]['generated_text'])

<s>[QUESTION] 트리플 바이패스 수술 [/QUESTION]트리플바이패스수술 [/INST] ['안녕하세요.하이닥외과상담의송슬기입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.트리플바이패스수술은 


In [10]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["트리플 바이패스 수술"]
references = ['안녕하세요.하이닥외과상담의송슬기입니다.트리플바이패스수술은 3가지수술을동시에하는수술입니다.']
results = bertscore.compute(predictions=predictions, references=references, lang="en")

ModuleNotFoundError: No module named 'evaluate'

In [28]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00


In [27]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
